In [1]:
import baostock as bs
import pandas as pd
from datetime import datetime, timedelta
from IPython.display import display, clear_output

# 登录 baostock
lg = bs.login()
if lg.error_code != '0':
    print(f"登录 baostock 失败，错误代码：{lg.error_code}, 错误信息：{lg.error_msg}")
else:
    print("BaoStock 初始化完成")

# 手动指定日期为2025/01/20
recent_trading_day = '20250120'
print(f"使用固定日期作为分析日期：{recent_trading_day}")

def get_all_stock_codes():
    """
    获取所有 A 股股票代码，剔除 ST、科创板、北交所股票以及指数。
    """
    rs = bs.query_stock_basic()
    data_list = []
    if rs.error_code != '0':
        print(f"查询股票基本信息失败，错误代码：{rs.error_code}, 错误信息：{rs.error_msg}")
        return [], []
    while rs.next():
        data_list.append(rs.get_row_data())
    stock_df = pd.DataFrame(data_list, columns=rs.fields)

    # 剔除 ST 股票
    stock_df = stock_df[~stock_df['code_name'].str.contains('ST', na=False)]
    # 剔除科创板股票（688开头）
    stock_df = stock_df[~stock_df['code'].str.startswith('sh.688')]
    # 剔除北交所股票（bj.开头）
    stock_df = stock_df[~stock_df['code'].str.startswith('bj.')]
    # 剔除非 A 股股票（type != '1'）
    stock_df = stock_df[stock_df['type'] == '1']

    codes = stock_df['code'].tolist()
    names = stock_df['code_name'].tolist()
    print(f"获取到 {len(codes)} 只股票代码。")
    return codes, names

codes, names = get_all_stock_codes()
if not codes:
    print("未获取到任何股票代码，程序终止。")
    bs.logout()
    exit()

def fetch_stock_data(code, start_date, end_date):
    """
    获取单只股票的历史数据
    """
    # 包含 turn 和 amount 字段
    fields = "date,code,open,high,low,close,volume,amount,turn"
    rs = bs.query_history_k_data_plus(code, fields, start_date=start_date, end_date=end_date, frequency="d", adjustflag="2")
    data_list = []
    if rs.error_code != '0':
        # 数据获取失败则打印信息并返回空df
        print(f"查询股票 {code} 历史数据失败，错误代码：{rs.error_code}, 错误信息：{rs.error_msg}")
        return pd.DataFrame()
    while rs.next():
        data_list.append(rs.get_row_data())
    data = pd.DataFrame(data_list, columns=rs.fields)
    
    # 将需要数值化的列转为数值类型
    for col in ["open", "high", "low", "close", "volume", "amount", "turn"]:
        data[col] = pd.to_numeric(data[col], errors='coerce')

    return data

def analyze_stock_bollinger_only(data, code, name, check_date):
    """
    判断股票在指定日期 check_date 是否满足以下条件：
    - 指定日期的最低价低于布林带下轨的100%。
    布林参数：20日均值，2倍标准差
    """
    if data.empty or len(data) < 20:
        return None

    data['date'] = pd.to_datetime(data['date'], errors='coerce')
    data = data.dropna(subset=['close', 'low'])

    if len(data) < 20:
        return None

    period = 20
    std_multiplier = 2
    # 计算布林带
    data['Middle Band'] = data['close'].rolling(window=period, min_periods=20).mean()
    data['STD'] = data['close'].rolling(window=period, min_periods=20).std()
    data['Upper Band'] = data['Middle Band'] + std_multiplier * data['STD']
    data['Lower Band'] = data['Middle Band'] - std_multiplier * data['STD']

    # 查找指定日期的数据行
    target_date = datetime.strptime(check_date, '%Y%m%d')
    target_row = data[data['date'] == target_date]
    if target_row.empty:
        return None

    recent_day = target_row.iloc[0]
    if pd.isna(recent_day['Lower Band']):
        return None

    # 修改条件为最低价低于布林带下轨的100%
    lower_band_threshold = recent_day['Lower Band'] * 1.02

    if recent_day['low'] < lower_band_threshold:
        return {
            "Code": code,
            "Name": name,
            "Date": recent_day['date'].strftime('%Y-%m-%d'),
            "Close": recent_day['close'],
            "Low": recent_day['low'],
            "Lower Band": recent_day['Lower Band'],
            "Threshold": lower_band_threshold,
            # 新增换手率和成交额
            "Turn": recent_day['turn'],
            "Amount": recent_day['amount']
        }

    return None


def run_analysis_bollinger_only(codes, names, recent_trading_day):
    """
    扫描股票列表，找出在指定 recent_trading_day 这天最低价低于布林带下轨的票。
    """
    # 回溯90天保证足够数据
    start_date = (datetime.strptime(recent_trading_day, '%Y%m%d') - timedelta(days=90)).strftime('%Y-%m-%d')
    end_date = datetime.strptime(recent_trading_day, '%Y%m%d').strftime('%Y-%m-%d')
    print(f"分析日期范围：{start_date} 至 {end_date}")

    all_results = pd.DataFrame(columns=["Code", "Name", "Date", "Close", "Low", "Lower Band", "Threshold", "Turn", "Amount"])
    total = len(codes)

    for idx, (code, name) in enumerate(zip(codes, names)):
        print(f"正在分析第 {idx + 1}/{total} 只股票：{code} - {name}")
        try:
            data = fetch_stock_data(code, start_date, end_date)
            if data.empty:
                continue
            result = analyze_stock_bollinger_only(data, code, name, recent_trading_day)
            if result:
                all_results = pd.concat([all_results, pd.DataFrame([result])], ignore_index=True)
                # 找到符合条件的股票后，实时更新显示
                clear_output(wait=True)
                display(all_results)

            # 每处理100只股票打印进度
            if (idx + 1) % 100 == 0:
                print(f"已处理 {idx + 1}/{total} 只股票，当前总记录数：{len(all_results)}")
        except Exception as e:
            print(f"股票 {code} 分析失败：{e}，跳过该股票。")

    # 最终显示结果
    clear_output(wait=True)
    display(all_results)
    print(f"分析完成，总共找到 {len(all_results)} 只符合布林带条件的股票（日期：{recent_trading_day}）。")
    return all_results

if codes:
    print("开始运行分析（2025/01/20最低价低于布林带下轨的股票）")
    signal_stocks_bollinger_df = run_analysis_bollinger_only(codes, names, recent_trading_day)
    print("符合条件的股票：")
    display(signal_stocks_bollinger_df)
    if not signal_stocks_bollinger_df.empty:
        # 修改最终保存的文件名
        signal_stocks_bollinger_df.to_excel("bollinger_signal_stocks_20250120_with_turn.xlsx", index=False)
        print("结果已保存为 bollinger_signal_stocks_20250120_with_turn.xlsx")
    else:
        print("无符合条件的股票。")

bs.logout()

,Code,Name,Date,Close,Low,Lower Band,Threshold,Turn,Amount
0,sh.600012,皖通高速,2025-01-20,16.32,16.08,15.815839,16.132156,0.4286,8.128382e+07
1,sh.600021,上海电力,2025-01-20,8.52,8.48,8.321511,8.487941,2.3736,5.314635e+08
2,sh.600028,中国石化,2025-01-20,6.17,6.16,6.090139,6.211942,0.1498,8.785708e+08
3,sh.600039,四川路桥,2025-01-20,7.01,7.00,6.947483,7.086433,0.2187,1.032233e+08
4,sh.600048,保利发展,2025-01-20,8.60,8.43,8.291643,8.457476,1.0393,1.067657e+09
...,...,...,...,...,...,...,...,...,...
115,sz.300989,蕾奥规划,2025-01-20,15.16,14.76,14.754119,15.049202,2.3890,5.439844e+07
116,sz.300998,宁波方正,2025-01-20,14.49,14.25,14.216886,14.501224,2.0786,3.252733e+07
117,sz.301062,上海艾录,2025-01-20,12.06,12.04,11.964632,12.203925,2.5139,9.609086e+07
118,sz.301305,朗坤环境,2025-01-20,18.02,17.28,16.989468,17.329257,2.0605,4.615599e+07


分析完成，总共找到 120 只符合布林带条件的股票（日期：20250120）。
符合条件的股票：


,Code,Name,Date,Close,Low,Lower Band,Threshold,Turn,Amount
0,sh.600012,皖通高速,2025-01-20,16.32,16.08,15.815839,16.132156,0.4286,8.128382e+07
1,sh.600021,上海电力,2025-01-20,8.52,8.48,8.321511,8.487941,2.3736,5.314635e+08
2,sh.600028,中国石化,2025-01-20,6.17,6.16,6.090139,6.211942,0.1498,8.785708e+08
3,sh.600039,四川路桥,2025-01-20,7.01,7.00,6.947483,7.086433,0.2187,1.032233e+08
4,sh.600048,保利发展,2025-01-20,8.60,8.43,8.291643,8.457476,1.0393,1.067657e+09
...,...,...,...,...,...,...,...,...,...
115,sz.300989,蕾奥规划,2025-01-20,15.16,14.76,14.754119,15.049202,2.3890,5.439844e+07
116,sz.300998,宁波方正,2025-01-20,14.49,14.25,14.216886,14.501224,2.0786,3.252733e+07
117,sz.301062,上海艾录,2025-01-20,12.06,12.04,11.964632,12.203925,2.5139,9.609086e+07
118,sz.301305,朗坤环境,2025-01-20,18.02,17.28,16.989468,17.329257,2.0605,4.615599e+07


结果已保存为 bollinger_signal_stocks_20250120_with_turn.xlsx
logout success!


In [2]:
import pandas as pd

# 读取之前保存的excel文件
df = pd.read_excel("bollinger_signal_stocks_20250120_with_turn.xlsx")

# 去除股票代码中的前缀 "sh." 或 "sz."
df['Code'] = df['Code'].str.replace(r'^(sh|sz)\.', '', regex=True)

# 保存清理后的文件
df.to_excel("布林底部—0120.xlsx", index=False)
print("清理后文件已保存为布林底部—0120.xlsx")

清理后文件已保存为布林底部—0120.xlsx


In [3]:
import pandas as pd

# 1. 读取已有的 excel 文件
df = pd.read_excel("bollinger_signal_stocks_20250120_with_turn.xlsx")

# 2. 筛选满足 Turn >= 1.8 且 Turn <= 15 且 Amount >= 7,000万 的行
df_filtered = df[(df['Turn'] >= 1.8) & (df['Turn'] <= 15) & (df['Amount'] >= 70000000)]

# 3. 去除股票代码中的前缀 "sh." 或 "sz."
df_filtered['Code'] = df_filtered['Code'].str.replace(r'^(sh|sz)\.', '', regex=True)

# 4. 保存到新的文件
df_filtered.to_excel("布林底部—0120-考虑换手成交额.xlsx", index=False)
print("清理后文件已保存为布林底部—0120-考虑换手成交额.xlsx")


清理后文件已保存为布林底部—0120-考虑换手成交额.xlsx


/var/folders/kf/z22rptk919q5_wpj26yt385r0000gn/T/ipykernel_4652/1311502286.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_filtered['Code'] = df_filtered['Code'].str.replace(r'^(sh|sz)\.', '', regex=True)
